In [2]:
# 新闻爬取xlxs文件目录：
NEWS_PATH = './data/'
#语料库文件路径：
#DATA_PATH = '../data/news_words.txt'
#标记文件路径
#LABEL_PATH = '../data/news_labels.txt'
#标记数据路径
LABEL_DIR = './data_used/'
# LDA Model 路径：
LDA_PATH = './model/lda.model'
# Dictionary 路径
DICT_PATH = './model/dictionary.txtdic'
# Random Forest 模型路径
RF_PATH = './model/rf.model'
#coding=utf-8  
import codecs  
from gensim import corpora  
from gensim.models import LdaModel  
from gensim.corpora import Dictionary  


import jieba
import csv
import numpy as np  
import os  
import pandas as pd
from sklearn.naive_bayes import GaussianNB  
from sklearn.externals import joblib

import time    
from sklearn import metrics    
import pickle as pickle    
import pandas as pd  
  
import random


import shutil
import time
import word_segmentation as ws
word_segmentor = ws.WordSegmentation()

# 由CSV生成语料库

* 读取csv文件
* 对于一行资讯进行分词
* 将分词结果写入新的txt文件

In [11]:
# 常量定义

#语料库文件路径：
DATA_PATH = './data/news_words.txt'

import shutil
import time
import pandas as pd
import word_segmentation as ws
word_segmentor = ws.WordSegmentation()


# write_news_words
# 输入 csv 文件的 file_path
# 操作：分词-写入DATA_FILE
def write_news_words(file_path):
    file_data = pd.read_csv(file_path)
    file_data.rename(columns={'标题':'title', '正文':'content'}, inplace = True)
    file = open(DATA_PATH,"a")
    try:
        for index, row in file_data.iterrows():
            #print(row.content)
            row_words = word_segmentor.word_segmentation(str(row.title) + str(row.content))
            file_words = " ".join(row_words)
            file.write(file_words + ' \n')
    except:
        print("Error reading news from the file:", file)
    file.close()
    

# 添加文件到语料库
# 输入：new_path 为新增的抓取csv数据的文件夹
# 执行：将csv文件的分词结果写入语料库文件
def append_csv2txt(new_path,old_path):
    files = os.listdir(new_path)
    print(files)
    
    for fname in files:
        fpath = new_path + fname
        if 'csv' in fpath:
            print(fpath)
            write_news_words(fpath)
            shutil.move(fpath,old_path)
            

In [12]:
# 将 news 文件夹中的csv文件读取存入语料库，并将csv文件移至 corpus 文件夹中

start_time = time.time()
append_csv2txt('./data/','./data_used/')
print('Loading CSV files took %fs!' % (time.time() - start_time)) 


['动力煤.csv', '黄玉米.csv', '热轧卷板.csv', '细木工板.csv', '黄金.csv', '中密度纤维板.csv', '镍.csv', '锌.csv', '白银.csv', '菜粕.csv', '玉米淀粉.csv', '国债期货.csv', '甲醇.csv', '菜籽.csv', '铅.csv', '天然橡胶.csv', '燃料油.csv', '大豆原油.csv', '苹果.csv', '聚氯乙烯.csv', '小麦.csv', '棉纱.csv', '鲜鸡蛋.csv', '棕榈油.csv', '中质含硫原油.csv', '一号棉花.csv', '股指期货.csv', '阴极铜.csv', '冶金焦炭.csv', '聚丙烯.csv', '锰硅.csv', '早籼稻.csv', '焦煤.csv', '螺纹钢.csv', '铝.csv', '平板玻璃.csv', 'PTA.csv', '晚籼稻.csv', '铁矿石.csv', '石油沥青.csv', '油菜籽.csv', '锡.csv', '硅铁.csv', '白糖.csv', '线型低密度聚乙烯.csv', '粳稻谷.csv']
./data/动力煤.csv
./data/黄玉米.csv
./data/热轧卷板.csv
./data/细木工板.csv
./data/黄金.csv
./data/中密度纤维板.csv
./data/镍.csv
./data/锌.csv
./data/白银.csv
./data/菜粕.csv
./data/玉米淀粉.csv
./data/国债期货.csv
./data/甲醇.csv
./data/菜籽.csv
./data/铅.csv
./data/天然橡胶.csv
./data/燃料油.csv
./data/大豆原油.csv
./data/苹果.csv
./data/聚氯乙烯.csv
./data/小麦.csv
./data/棉纱.csv
./data/鲜鸡蛋.csv
./data/棕榈油.csv
./data/中质含硫原油.csv
./data/一号棉花.csv
./data/股指期货.csv
./data/阴极铜.csv
./data/冶金焦炭.csv
./data/聚丙烯.csv
./data/锰硅.csv
./data/早籼稻.csv
./data/焦煤.c

# 标记数据集生成

对标记数据csv进行读取-分词 的转换，并将结果连同label一起写入txt文件


In [2]:

# write_news_words
# 输入 csv 文件的 file_path
# 操作：分词-写入DATA_FILE
def write_news_labels(fpath):
    print("Reading label csv file:",fpath)
    file_data = pd.read_csv(fpath)
    file_data.rename(columns={'标题':'title', '正文':'content','正文1':'content',"字段1_文本":"title","关键词":"plate"}, inplace = True)
    file = open(LABEL_PATH,"a")
    try:
        for index, row in file_data.iterrows():
            #print(row.content)
            #news_word = jieba_split(str(row.title) + str(row.content))
            news_word = " ".join(word_segmentor.word_segmentation(str(row.title) + str(row.content)))
            #news_bow = dictionary.doc2bow(news_word)      #文档转换成bow  
            #news_lda = lda2list(lda[news_bow],301) #得到lda向量
            file.write(row.plate + "," + news_word + "\n")
            #print(row.plate,news_lda)
    except:
        print("Error reading labels from the file:", fpath)
    file.close()
    

# 添加文件到语料库
# 输入：new_path 为新增的抓取csv数据的文件夹
# 执行：将csv文件的分词结果写入语料库文件
def append_labels2txt(new_path,old_path):
    files = os.listdir(new_path)
    print(files)
    for fname in files:
        fpath = new_path + fname
        if 'csv' in fpath:
            print(fpath)
            write_news_labels(fpath)
            shutil.move(fpath,old_path)
            

In [3]:

start_time = time.time()
append_labels2txt('../labels/','../label_old/')
print('Loading CSV files took %fs!' % (time.time() - start_time)) 


['东财关键词对应新闻-0426-6.csv', '东方财经关键词板块对应新闻-0425-2.csv', '东方财经关键词板块对应新闻-0425-1.csv', '东财关键词对应新闻-0427.csv', '行业研究报告数据中心东方财富网-0502-2.csv', '行业研究报告数据中心东方财富网-0502-1.csv', '东财关键词对应新闻-0426-7.csv', '东财关键词对应新闻-0426-4.csv', '东财关键词对应新闻-0426-3.csv', '东财关键词对应新闻-0426-8.csv', '东方财经关键词板块对应新闻-0427.csv', '东财关键词对应新闻-0426-1.csv', '东财关键词对应新闻-0426-5.csv', '东财关键词对应新闻-0426-2.csv']
../labels/东财关键词对应新闻-0426-6.csv
Reading label csv file: ../labels/东财关键词对应新闻-0426-6.csv


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.890 seconds.
Prefix dict has been built succesfully.


Error reading labels from the file: ../labels/东财关键词对应新闻-0426-6.csv


Error: Destination path '../label_old/东财关键词对应新闻-0426-6.csv' already exists

# 离线语料分析建模及板块预测模型优化

LDA：

* 生成语料库（初期由csv文件生成，后期由数据集中获取）
* 读取语料库（每行一条资讯，已分词，空格分隔）
* 利用语料库训练LDA模型并保存

RF：

* 读取标记数据库（每行一条资讯，第一列为标签，第二列为资讯，已分词，空格分隔）
* 利用LDA模型转换资讯为Vector-标记 格式
* 训练RF模型并保存

## 语料库载入及LDA模型训练

In [13]:
class MyCorpus(object):
    def __iter__(self):
        for line in open(DATA_PATH):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.split(' '))

In [14]:
# 遍历语料库文件，逐步增加dictionary
print("Start reading corpus file...")
start_time = time.time()    
fr=open(DATA_PATH,'r')  
train=[]  
corpus_mem_friendly = MyCorpus()
# dictionary = corpora.Dictionary(corpus_mem_friendly)
dictionary = corpora.Dictionary(line.split(' ') for line in open(DATA_PATH))
print('Composing dictionary took %fs!' % (time.time() - start_time)) 
start_time = time.time()
lda = LdaModel(corpus=corpus_mem_friendly, id2word=dictionary, num_topics=400)  
lda.save(LDA_PATH)
print('LDA training took %fs!' % (time.time() - start_time)) 


Start reading corpus file...
Composing dictionary took 4.470590s!


/usr/local/python3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


LDA training took 197.254196s!


In [15]:
dictionary.save(DICT_PATH)

In [16]:
# 打印LDA模型相关结果
topic_list=lda.print_topics(40)  

for topic in topic_list:  
    print(topic) 
print("第一主题")
print(lda.print_topic(1))

(189, '0.043*"突破" + 0.041*"指标" + 0.036*"趋势" + 0.027*"中期" + 0.024*"意味" + 0.023*"意味着" + 0.019*"站上" + 0.019*"比率" + 0.017*"均线" + 0.017*"短期"')
(63, '0.444*"加价" + 0.061*"背靠" + 0.028*"不济" + 0.024*"供给量" + 0.022*"调节" + 0.015*"据卓创" + 0.013*"渐入" + 0.007*"惯性" + 0.005*"内地" + 0.005*"价格"')
(331, '0.247*"美金" + 0.060*"相比之下" + 0.053*"炼油" + 0.041*"追究" + 0.020*"三方" + 0.019*"之下" + 0.019*"美元" + 0.015*"原油" + 0.013*"公布" + 0.013*"止损"')
(107, '0.088*"均线" + 0.046*"振荡" + 0.029*"线" + 0.029*"指标" + 0.027*"呈现" + 0.026*"上行" + 0.019*"排列" + 0.018*"突破" + 0.016*"天" + 0.016*"短期"')
(49, '0.266*"石化" + 0.066*"大庆" + 0.041*"扬子" + 0.034*"大庆石化" + 0.030*"扬子石化" + 0.026*"责任" + 0.025*"\n" + 0.025*"编辑" + 0.025*"责任编辑" + 0.024*"价格"')
(224, '0.025*"图中" + 0.017*"日" + 0.014*"月" + 0.010*"跌到" + 0.010*"黄金" + 0.008*"反弹" + 0.008*"房市" + 0.007*"时差" + 0.007*"三波" + 0.007*"纤维"')
(58, '0.121*"库存" + 0.031*"依托" + 0.031*"氛围" + 0.025*"地区" + 0.020*"铁精粉" + 0.019*"焦化" + 0.019*"仍存" + 0.017*"加重" + 0.014*"区间" + 0.013*"厂"')
(338, '0.881*"食糖" + 0.007*"开仓" + 0.00

# 标签库载入及分类器训练

* 标记数据来源： 第一财经新闻等按关键词搜索的新闻爬取结果
* 标记格式：新闻标题/正文/关键词

训练过程：

* 读取含有“关键词”-“标题”-“正文” 的csv文件
* 对文件中的 标题 + 正文 字符串进行分词
* 分词结果使用LDA模型向量化
* 向量 ： 关键词 作为 Random Forest模型的训练输入



In [19]:
start_time = time.time()
lda = LdaModel.load(LDA_PATH)
print('Loading LDA model took %fs!' % (time.time() - start_time)) 

Loading LDA model took 0.125974s!


In [20]:
import word_segmentation
# 对于label文件夹下的CSV，读取后将标题+正文转换为分词结果，再转换为LDA结果，形成200维向量
# 对于新闻的标签，直接保留其表格中的“关键词”地段

corpus_mem_friendly = MyCorpus()
# dictionary = corpora.Dictionary(corpus_mem_friendly)
dictionary = corpora.Dictionary(line.split(' ') for line in open(DATA_PATH))

#df_label = pd.DataFrame(columns=['words','label'])  
news_vec = []
news_labels = []

# turn lda result into list
def lda2list(lda,topic_n):
    lda_dict = dict(lda)
    lda_list = [0] * topic_n
    for i in range(topic_n):
        lda_list[i] = lda_dict.get(i,0)
    return lda_list


LABEL_DIR = './data_used/'
start_time = time.time()
print("Start loading labeled data...")
files = os.listdir(LABEL_DIR)
print(files)
for fname in files:
    fpath = LABEL_DIR + fname
    if 'csv' in fpath:
        print(fpath)
        row_plate = fname.replace(".csv", "")
        print(row_plate)
        file_data = pd.read_csv(fpath)
        file_data.rename(columns={'标题':'title', '正文':'content'}, inplace = True)
        for index, row in file_data.iterrows():
            #print(row.content)
            #news_word = jieba_split(str(row.title) + str(row.content))
            news_word = word_segmentor.word_segmentation(str(row.title) + str(row.content))
            news_bow = dictionary.doc2bow(news_word)      #文档转换成bow  
            news_lda = lda2list(lda[news_bow],301) #得到lda向量
            news_vec.append(news_lda)
            news_labels.append(row_plate)
            #print(row.plate,news_lda)
    print("done")
print('Loading labeled data took %fs!' % (time.time() - start_time)) 
        

Start loading labeled data...
['动力煤.csv', '黄玉米.csv', '热轧卷板.csv', '细木工板.csv', '黄金.csv', '中密度纤维板.csv', '镍.csv', '锌.csv', '白银.csv', '菜粕.csv', '玉米淀粉.csv', '国债期货.csv', '甲醇.csv', '菜籽.csv', '铅.csv', '天然橡胶.csv', '燃料油.csv', '大豆原油.csv', '苹果.csv', '聚氯乙烯.csv', '小麦.csv', '棉纱.csv', '鲜鸡蛋.csv', '棕榈油.csv', '中质含硫原油.csv', '一号棉花.csv', '股指期货.csv', '阴极铜.csv', '冶金焦炭.csv', '聚丙烯.csv', '锰硅.csv', '早籼稻.csv', '焦煤.csv', '螺纹钢.csv', '铝.csv', '平板玻璃.csv', 'PTA.csv', '晚籼稻.csv', '铁矿石.csv', '石油沥青.csv', '油菜籽.csv', '锡.csv', '硅铁.csv', '白糖.csv', '线型低密度聚乙烯.csv', '粳稻谷.csv']
./data_used/动力煤.csv
动力煤
done
./data_used/黄玉米.csv
黄玉米
done
./data_used/热轧卷板.csv
热轧卷板
done
./data_used/细木工板.csv
细木工板
done
./data_used/黄金.csv
黄金
done
./data_used/中密度纤维板.csv
中密度纤维板
done
./data_used/镍.csv
镍
done
./data_used/锌.csv
锌
done
./data_used/白银.csv
白银
done
./data_used/菜粕.csv
菜粕
done
./data_used/玉米淀粉.csv
玉米淀粉
done
./data_used/国债期货.csv
国债期货
done
./data_used/甲醇.csv
甲醇
done
./data_used/菜籽.csv
菜籽
done
./data_used/铅.csv
铅
done
./data_used/天然橡胶.csv
天然橡胶
done
./da

In [21]:
   
# Random Forest Classifier    
def random_forest_classifier(train_x, train_y):    
    from sklearn.ensemble import RandomForestClassifier    
    model = RandomForestClassifier(n_estimators=8)    
    model.fit(train_x, train_y)    
    return model    
    

X = np.array([np.array(xi) for xi in news_vec])
Y = np.array(news_labels)

print("********classifier: random forest ***********")
start_time = time.time()    
model = random_forest_classifier(X, Y)
print('training took %fs!' % (time.time() - start_time)) 
for i in range(100):
    n = random.randrange(1,len(X))
    # predict_proba(x)：给出带有概率值的结果。每个点在所有label的概率和为1.  
    # predict(x)：直接给出预测结果。内部还是调用的predict_proba()，根据概率的结果看哪个类型的预测值最高就是哪个类型。  
    print(model.predict([X[n]]))
    print(Y[n])

********classifier: random forest ***********
training took 1.081721s!
['白糖']
白糖
['甲醇']
甲醇
['铁矿石']
铁矿石
['黄玉米']
黄玉米
['一号棉花']
一号棉花
['小麦']
小麦
['棕榈油']
棕榈油
['燃料油']
燃料油
['股指期货']
股指期货
['动力煤']
动力煤
['黄金']
黄金
['小麦']
小麦
['燃料油']
燃料油
['动力煤']
动力煤
['天然橡胶']
天然橡胶
['棕榈油']
棕榈油
['燃料油']
燃料油
['硅铁']
硅铁
['铁矿石']
铁矿石
['苹果']
苹果
['聚氯乙烯']
聚氯乙烯
['小麦']
小麦
['黄玉米']
黄玉米
['聚氯乙烯']
聚氯乙烯
['小麦']
小麦
['鲜鸡蛋']
鲜鸡蛋
['大豆原油']
大豆原油
['聚氯乙烯']
聚氯乙烯
['锌']
锌
['螺纹钢']
螺纹钢
['阴极铜']
阴极铜
['菜粕']
菜粕
['棉纱']
一号棉花
['螺纹钢']
天然橡胶
['黄玉米']
黄玉米
['白糖']
白糖
['阴极铜']
阴极铜
['一号棉花']
一号棉花
['聚氯乙烯']
聚氯乙烯
['白糖']
白糖
['甲醇']
甲醇
['黄金']
黄金
['甲醇']
甲醇
['棉纱']
棉纱
['棉纱']
棉纱
['小麦']
小麦
['一号棉花']
一号棉花
['燃料油']
燃料油
['铁矿石']
铁矿石
['镍']
镍
['冶金焦炭']
冶金焦炭
['白银']
白银
['棉纱']
棉纱
['热轧卷板']
热轧卷板
['中质含硫原油']
中质含硫原油
['PTA']
PTA
['一号棉花']
一号棉花
['黄金']
黄金
['镍']
镍
['黄金']
黄金
['粳稻谷']
粳稻谷
['硅铁']
硅铁
['锌']
锌
['铅']
铅
['中质含硫原油']
中质含硫原油
['大豆原油']
大豆原油
['一号棉花']
一号棉花
['阴极铜']
阴极铜
['一号棉花']
一号棉花
['小麦']
小麦
['白糖']
白糖
['阴极铜']
阴极铜
['棕榈油']
棕榈油
['动力煤']
动力煤
['螺纹钢']
天然橡胶
['螺纹钢']
油菜籽
['黄金']
黄金
['黄金']
黄金
['小麦']
小麦
['白糖']
白糖
[

In [22]:
for i in range(100):
    n = random.randrange(1,len(X))
    # predict_proba(x)：给出带有概率值的结果。每个点在所有label的概率和为1.  
    # predict(x)：直接给出预测结果。内部还是调用的predict_proba()，根据概率的结果看哪个类型的预测值最高就是哪个类型。  
    print([X[n]])
    print(model.predict([X[n]]))
    print(Y[n])

[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03078425, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01731808, 0.        , 0.        , 0.        , 0.    

['棕榈油']
菜籽
[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.16706599, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.50562561,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01575727, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.     

['阴极铜']
阴极铜
[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.04161306, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01174875, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.    

['镍']
镍
[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.23736742, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.0192425 , 0.        , 0.        

In [23]:
joblib.dump(model, RF_PATH)

['./model/rf.model']